###### Imports and Settings

In [1]:
import pandas as pd
import numpy as np
import requests
#import io
import pickle
from collections import deque
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
pd.options.mode.chained_assignment = None  # default='warn'

# Comprehensive Plan Data Pull

In [2]:
#to read in... rb is read bite
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)

In [3]:
#create a variable that contains your api key
api_key = keys_dict_2['CENSUS']
#api_key = '24fc7d81b74510d599f702dbd408fb18e1466d81'

In [4]:
GNRC = ['111', #Macon
       '161', #Stewart
       '125', #Montgomery
       '083', #Houston
       '085', #Humphreys
       '043', #Dickson
       '021', #Cheatham
       '147', #Robertson
       '165', #Sumner
       '037', #Davidson
       '189', #Wilson
       '169', #Trousdale
       '187', #Williamson
       '149', #Rutherford
       '119'] #Maury
KY = ['003', #Allen
      '213'] #Simpson
qlaces = ['1600000US4702180', #Ashland City: Cheatham
          '1600000US4739660', #Kingston Springs: Cheatham
          '1600000US4757480', #Pegram: Cheatham
          '1600000US4759560', #Pleasant View: Cheatham
          '1600000US4704620', #Belle Meade: Davidson
          '1600000US4705140', #Berry Hill: Davidson
          '1600000US4727020', #Forest Hills: Davidson
          '1600000US4729920', #Goodlettsville: Davidson/Sumner
          '1600000US4752006', #Nashville-Davidson metropolitan government (balance): Davidson
          '1600000US4754780', #Oak Hill: Davidson
          '1600000US4763140', #Ridgetop: Davidson/Robertson
          '1600000US4709880', #Burns: Dickson
          '1600000US4713080', #Charlotte: Dickson
          '1600000US4720620', #Dickson: Dickson
          '1600000US4769080', #Slayden: Dickson
          '1600000US4776860', #Vanleer: Dickson
          '1600000US4779980', #White Bluff: Dickson
          '1600000US4724320', #Erin: Houston
          '1600000US4773460', #Tennessee Ridge: Houston/Stewart
          '1600000US4744840', #McEwen: Humphreys
          '1600000US4752820', #New Johnsonville: Humphreys
          '1600000US4778560', #Waverly: Humphreys
          '1600000US4716540', #Columbia: Maury
          '1600000US4751080', #Mount Pleasant: Maury
          '1600000US4770580', #Spring Hill: Maury/Williamson
          '1600000US4715160', #Clarksville: Montgomery
          '1600000US4700200', #Adams: Robertson
          '1600000US4711980', #Cedar Hill: Robertson
          '1600000US4716980', #Coopertown: Robertson
          '1600000US4718420', #Cross Plains: Robertson
          '1600000US4730960', #Greenbrier: Robertson
          '1600000US4748980', #Millersville: Robertson/Sumner
          '1600000US4760280', #Portland: Robertson/Sumner
          '1600000US4770500', #Springfield: Robertson
          '1600000US4780200', #White House: Robertson/Sumner
          '1600000US4722360', #Eagleville: Rutherford
          '1600000US4741200', #La Vergne: Rutherford
          '1600000US4751560', #Murfreesboro: Rutherford
          '1600000US4769420', #Smyrna: Rutherford
          '1600000US4718820', #Cumberland City: Stewart
          '1600000US4721400', #Dover: Stewart
          '1600000US4728540', #Gallatin: Sumner
          '1600000US4733280', #Hendersonville: Sumner
          '1600000US4779420', #Westmoreland: Sumner
          '1600000US4708280', #Brentwood: Williamson
          '1600000US4725440', #Fairview: Williamson
          '1600000US4727740', #Franklin: Williamson
          '1600000US4753460', #Nolensville: Williamson
          '1600000US4773900', #Thompson's Station: Williamson
          '1600000US4741520', #Lebanon: Wilson
          '1600000US4750780', #Mount Juliet: Wilson
          '1600000US4778320', #Watertown: Wilson
          '1600000US4740160', #Lafayette: Macon
          '1600000US2169114', #Scottsville city, Kentucky
          '1600000US2128918', #Franklin city, Kentucky
          '1600000US4749460'] #Mitchelville city: Sumner

## Read In Data Guide

The "head" should never be more than 2 variables, and the "tail" never more than 2. Since we can pull 50 variables at once this means that we can systematically program in 46 variables for each pull, so that's:
+ dg1: ID's 1 - 46  
+ dg2: ID's 47-92  
+ dg3: ID's 93-138  
+ dg4: ID's 139-184  
+ dg5: ID's 185-230  
+ dg6: ID's 231-276  
+ dg7: ID's 277-322  
+ dg8: ID's 323-368  
+ dg9: ID's 369-414  
+ dg10: ID's 415-460  
+ dg11: ID's 461-506  
+ dg12: ID's 507-552  
+ dg13: ID's 553-598  

**This data guide stops at ID 554 as of 7/8/2022.**

In [5]:
dataguide = pd.read_csv('../../data guides/DATA GUIDE ACS 2010.csv', dtype = str)
dataguide['ID'] = dataguide['ID'].astype(int)

In [6]:
dg1 = dataguide[dataguide['ID'].between(1, 46)]
dg2 = dataguide[dataguide['ID'].between(47, 92)]
dg3 = dataguide[dataguide['ID'].between(93, 138)]
dg4 = dataguide[dataguide['ID'].between(139, 184)]
dg5 = dataguide[dataguide['ID'].between(185, 230)]
dg6 = dataguide[dataguide['ID'].between(231, 276)]
dg7 = dataguide[dataguide['ID'].between(277, 322)]
dg8 = dataguide[dataguide['ID'].between(323, 368)]
dg9 = dataguide[dataguide['ID'].between(369, 414)]
dg10 = dataguide[dataguide['ID'].between(415, 460)]
dg11 = dataguide[dataguide['ID'].between(461, 506)]
dg12 = dataguide[dataguide['ID'].between(507, 552)]
dg13 = dataguide[dataguide['ID'].between(553, 598)]

## One Through Twenty Seven

In [7]:
# ONE
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg1['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg1['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
onepull = df
print('Okay Finished')

Okay Finished


In [8]:
#placemarker
one = onepull

In [9]:
#TWO
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg2['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg2['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
twopull = df
print('Okay Finished')

Okay Finished


In [11]:
two = twopull

In [12]:
# THREE
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg3['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg3['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
threepull = df
print('Okay Finished')

Okay Finished


In [13]:
three = threepull

In [14]:
# FOUR
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg4['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg4['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
fourpull = df
print('Okay Finished')

Okay Finished


In [15]:
four = fourpull

In [16]:
# FIVE
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg5['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg5['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
fivepull = df
print('Okay Finished')

Okay Finished


In [17]:
five = fivepull

In [18]:
# SIX
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg6['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg6['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
sixpull = df
print('Okay Finished')

Okay Finished


In [19]:
six = sixpull

In [20]:
# SEVEN
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg7['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg7['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
sevenpull = df
print('Okay Finished')

Okay Finished


In [21]:
seven = sevenpull

In [22]:
# EIGHT
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg8['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg8['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
eightpull = df
print('Okay Finished')

Okay Finished


In [23]:
eight = eightpull

In [24]:
# NINE
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg9['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg9['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
ninepull = df
print('Okay Finished')

Okay Finished


In [25]:
nine = ninepull

In [26]:
# TEN
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg10['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg10['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
tenpull = df
print('Okay Finished')

Okay Finished


In [27]:
ten = tenpull

In [28]:
# ELEVEN
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg11['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg11['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
elevenpull = df
print('Okay Finished')

Okay Finished


In [29]:
eleven = elevenpull

In [30]:
# TWELVE
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg12['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg12['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
twelvepull = df
print('Okay Finished')

Okay Finished


In [31]:
twelve = twelvepull

In [32]:
# THIRTEEN
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg13['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg13['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
thirteenpull = df
print('Okay Finished')

Okay Finished


In [33]:
thirteen = thirteenpull

In [34]:
last = thirteen

In [35]:
dfs = [two,three,four,five,six,seven,eight,nine,ten,eleven,twelve]

In [36]:
for df in dfs:
    df = df.drop(columns = ['NAME','StateFIPS','GeoFIPS'], inplace = True)

In [37]:
one = one.drop(columns = ['StateFIPS','GeoFIPS'])
last = last.drop(columns = 'NAME')

In [38]:
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['GEO_ID'],
                                            how='outer'), dfs)

In [39]:
dfs = [one, df_merged, last]
data = reduce(lambda  left,right: pd.merge(left,right,on=['GEO_ID'],
                                            how='outer'), dfs)

In [40]:
data = data.set_index(['NAME', 'GEO_ID'])
data.head()

,,agebysex_total_series,hhsize_avg,poverty_total_bysexbyage_series,poverty_belowlevel,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,tenure_total_series,tenure_owneroccunits,tenure_renteroccunits,units_total_series,units_one_detached,units_one_attached,units_two,units_threetofour,units_fivetonine,units_tentonineteen,units_twentytofortynine,units_fiftyormore,units_mobilehome,units_boatrvvanetc,hhtype_total_series,hhtype_familyhh,hhtype_familyhh_marriedcouplefam,hhtype_familyhh_otherfam,hhtype_familyhh_malenospouse,hhtype_familyhh_femalenospouse,hhtype_nonfamhh,hhtype_nonfamhh_householderalone,hhtype_nonfamhh_householdernotalone,hhincome_median,hhincome_total_series,hhincome_lessthan10000,hhincome_10to14999,hhincome_15to19999,hhincome_20to24999,hhincome_25to29999,hhincome_30to34999,hhincome_35to39999,hhincome_40to44999,hhincome_45to49999,hhincome_50to59999,hhincome_60to74999,hhincome_75to99999,hhincome_100to124999,hhincome125to149999,hhincome150to199999,hhincome200ormore,housingcost_total_selectedownercosts%hhincome_series,housingcost_total%ownercostwmortgage_series,housingcost_%ownercost30to34.9_wmortgage,housingcost_%ownercost35to39.9_wmortgage,housingcost_%ownercost40to49.9_wmortgage,housingcost_%ownercost50+_wmortgage,housingcost_total%ownercostwomortgage_series,housingcost_%ownercost30to34.9_womortgage,housingcost_%ownercost35to39.9_womortgage,housingcost_%ownercost40to49.9_womortgage,housingcost_%ownercost50+_womortgage,housingcost_total_rent%hhincome_series,housingcost_%rentercost30to34.9,housingcost_%rentercost35to39.9,housingcost_%rentercost40to49.9,housingcost_%rentercost50+,housingcost_medvalue_ownerocc_wmortgage,housingcost_medvalue_ownerocc_womortgage,housingcost_mediangrossrent_renteroccupied,commute_total_meansoftransportationtowork_series,commute_cartruckvan,commute_cartruckvan_drovealone,commute_cartruckvan_carpooled,commute_cartruckvan_carpooled_2ppl,commute_cartruckvan_carpooled_3ppl,commute_cartruckvan_carpooled_4ormoreppl,commute_publictransportation,commute_publictransportation_bus,commute_publictransportation_subwayorelevatedrail,commute_publictransportation_longdistancetrainorcommuterrail,commute_publictransportation_lightrailstreetcarortrolley,commute_publictransportation_ferryboat,commute_bicycle,commute_walk,commute_taxicabmotorcycleother,commute_workedfromhome,foreignborn_total,fb_europe,fb_asia,fb_africa,fb_oceania,fb_americas,fb_am_latinamerica,fb_am_northern,classworker_total_series,classworker_privateforprofit,classworker_privateforprofit_employee,classworker_privateforprofit_selfemployedincorporatedbusiness,classworker_privatenotforprofit,classworker_localgovt,classworker_stategovt,classworker_federalgovt,classworker_selfemployednonincorporatedbusiness,classworker_unpaidfamilyworker,lfstatus_total_sexbyagebyemploymentstatus16+_series,lfstatus_mciv_16to19,lfstatus_mciv_16to19_employed,lfstatus_mciv_16to19_unemployed,lfstatus_mciv_20to21,lfstatus_mciv_20to21_employed,lfstatus_mciv_20to21_unemployed,lfstatus_mciv_22to24,lfstatus_mciv_22to24_employed,lfstatus_mciv_22to24_unemployed,lfstatus_mciv_25to29,lfstatus_mciv_25to29_employed,lfstatus_mciv_25to29_unemployed,lfstatus_mciv_30to34,lfstatus_mciv_30to34_employed,lfstatus_mciv_30to34_unemployed,lfstatus_mciv_35to44,lfstatus_mciv_35to44_employed,lfstatus_mciv_35to44_unemployed,lfstatus_mciv_45to54,lfstatus_mciv_45to54_employed,lfstatus_mciv_45to54_unemployed,lfstatus_mciv_55to59,lfstatus_mciv_55to59_employed,lfstatus_mciv_55to59_unemployed,lfstatus_mciv_60to61,lfstatus_mciv_60to61_employed,lfstatus_mciv_60to61_unemployed,lfstatus_mciv_62to64,lfstatus_mciv_62to64_employed,lfstatus_mciv_62to64_unemployed,lfstatus_mciv_65to69,lfstatus_mciv_65to69_employed,lfstatus_mciv_65to69_unemployed,lfstatus_mciv_70to74,lfstatus_mciv_70to74_employed,lfstatus_mciv_70to74_unemployed,lfstatus_mciv_75+,lfstatus_mciv_75+_employed,lfstatus_mciv_75+_unemployed,lfstatus_fciv_16to19,lfstatus_fciv_16to19_employed,lfstatus_fciv_16to19_unem

In [41]:
transp = data.transpose()
transp.head()

NAME,"Cheatham County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Macon County, Tennessee","Maury County, Tennessee","Montgomery County, Tennessee","Robertson County, Tennessee","Rutherford County, Tennessee","Stewart County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee","Allen County, Kentucky","Simpson County, Kentucky","Franklin city, Kentucky","Scottsville city, Kentucky","Adams city, Tennessee","Ashland City town, Tennessee","Belle Meade city, Tennessee","Berry Hill city, Tennessee","Brentwood city, Tennessee","Burns town, Tennessee","Cedar Hill city, Tennessee","Charlotte town, Tennessee","Clarksville city, Tennessee","Columbia city, Tennessee","Coopertown town, Tennessee","Cross Plains city, Tennessee","Cumberland City town, Tennessee","Dickson city, Tennessee","Dover city, Tennessee","Eagleville city, Tennessee","Erin city, Tennessee","Fairview city, Tennessee","Forest Hills city, Tennessee","Franklin city, Tennessee","Gallatin city, Tennessee","Goodlettsville city, Tennessee","Greenbrier town, Tennessee","Hendersonville city, Tennessee","Kingston Springs town, Tennessee","Lafayette city, Tennessee","La Vergne city, Tennessee","Lebanon city, Tennessee","McEwen city, Tennessee","Millersville city, Tennessee","Mitchellville city, Tennessee","Mount Juliet city, Tennessee","Mount Pleasant city, Tennessee","Murfreesboro city, Tennessee","Nashville-Davidson metropolitan government (balance), Tennessee","New Johnsonville city, Tennessee","Nolensville town, Tennessee","Oak Hill city, Tennessee","Pegram town, Tennessee","Pleasant View city, Tennessee","Portland city, Tennessee","Ridgetop city, Tennessee","Slayden town, Tennessee","Smyrna town, Tennessee","Springfield city, Tennessee","Spring Hill city, Tennessee","Tennessee Ridge town, Tennessee","Thompson's Station town, Tennessee","Vanleer town, Tennessee","Watertown city, Tennessee","Waverly city, Tennessee","Westmoreland town, Tennessee","White Bluff town, Tennessee","White House city, Tennessee",Tennessee,United States
GEO_ID,0500000US47021,0500000US47037,0500000US47043,0500000US47083,0500000US47085,0500000US47111,0500000US47119,0500000US47125,0500000US47147,0500000US47149,0500000US47161,0500000US47165,0500000US47169,0500000US47187,0500000US47189,0500000US21003,0500000US21213,1600000US2128918,1600000US2169114,1600000US4700200,1600000US4702180,1600000US4704620,1600000US4705140,1600000US4708280,1600000US4709880,1600000US4711980,1600000US4713080,1600000US4715160,1600000US4716540,1600000US4716980,1600000US4718420,1600000US4718820,1600000US4720620,1600000US4721400,1600000US4722360,1600000US4724320,1600000US4725440,1600000US4727020,1600000US4727740,1600000US4728540,1600000US4729920,1600000US4730960,1600000US4733280,1600000US4739660,1600000US4740160,1600000US4741200,1600000US4741520,1600000US4744840,1600000US4748980,1600000US4749460,1600000US4750780,1600000US4751080,1600000US4751560,1600000US4752006,1600000US4752820,1600000US4753460,1600000US4754780,1600000US4757480,1600000US4759560,1600000US4760280,1600000US4763140,1600000US4769080,1600000US4769420,1600000US4770500,1600000US4770580,1600000US4773460,1600000US4773900,1600000US4776860,1600000US4778320,1600000US4778560,1600000US4779420,1600000US4779980,1600000US4780200,0400000US47,0100000US
agebysex_total_series,38809,612884,48712,8286,18416,21934,79029,163603,64347,250517,13133,155592,7751,174260,109563,19688,17214,8368,4251,565,4405,2891,467,35331,1634,391,1104,126148,34482,4060,1639,350,14299,1659,460,1538,7500,4769,59905,29084,15448,6195,49725,2787,4383,30367,25188,1649,6111,211,21922,4555,102602,587790,1793,5351,4498,2300,3962,10973,1855,252,38102,16097,25415,1389,1946,405,1616,4080,2555,3077,9764,6234968,303965272
hhsize_avg,2.67,2.34,2.57,2.40,2.43,2.61,2.45,2.64,2.69,2.65,2.44,2.60,2.75,2.78,2.63,2.47,2.52,2.45,2.22,2.47,2.38,2.71,1.47,2.99,2.43,2.98,2.57,2.63,2.3

In [42]:
numcols = list(transp.columns)
numcols
transp[numcols] = transp[numcols].astype(float)

In [43]:
data = transp

In [44]:
GNRCCounties = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                data[('Rutherford County, Tennessee', '0500000US47149')]]
data['GNRC'] = sum(GNRCCounties)
GNRCCountiesAll = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                   data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                   data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                   data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                   data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                   data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                   data[('Rutherford County, Tennessee', '0500000US47149')],data[('Maury County, Tennessee', '0500000US47119')]]
data['GNRC Region'] = sum(GNRCCountiesAll)
MPOCounties = [data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
               data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
               data[('Williamson County, Tennessee', '0500000US47187')],data[('Rutherford County, Tennessee', '0500000US47149')],
               data[('Maury County, Tennessee', '0500000US47119')]]
data['MPO'] = sum(MPOCounties)
RuthInc = [data[('Eagleville city, Tennessee', '1600000US4722360')],data[('La Vergne city, Tennessee', '1600000US4741200')],
           data[('Murfreesboro city, Tennessee', '1600000US4751560')],data[('Smyrna town, Tennessee', '1600000US4769420')]]
data[('Rutherford Incorporated', 'None')] = sum(RuthInc)
data[('Rutherford Unincorporated', 'None')] = data[('Rutherford County, Tennessee', '0500000US47149')] - data[('Rutherford Incorporated', 'None')]
WilsonInc = [data[('Lebanon city, Tennessee', '1600000US4741520')],data[('Mount Juliet city, Tennessee', '1600000US4750780')],
             data[('Watertown city, Tennessee', '1600000US4778320')]]
data[('Wilson Incorporated', 'None')] = sum(WilsonInc)
data[('Wilson Unincorporated', 'None')] = data[('Wilson County, Tennessee', '0500000US47189')] - data[('Wilson Incorporated', 'None')]
CheathInc = [data[('Ashland City town, Tennessee', '1600000US4702180')],data[('Kingston Springs town, Tennessee', '1600000US4739660')],
             data[('Pegram town, Tennessee', '1600000US4757480')],data[('Pleasant View city, Tennessee', '1600000US4759560')]]
data[('Cheatham Incorporated', 'None')] = sum(CheathInc)
data[('Cheatham Unincorporated', 'None')] = data[('Cheatham County, Tennessee', '0500000US47021')] - data[('Cheatham Incorporated', 'None')]
DicksInc = [data[('Burns town, Tennessee', '1600000US4709880')],data[('Charlotte town, Tennessee', '1600000US4713080')],
            data[('Dickson city, Tennessee', '1600000US4720620')],data[('Slayden town, Tennessee', '1600000US4769080')],
            data[('Vanleer town, Tennessee', '1600000US4776860')],data[('White Bluff town, Tennessee', '1600000US4779980')]]
data[('Dickson Incorporated', 'None')] = sum(DicksInc)
data[('Dickson Unincorporated', 'None')] = data[('Dickson County, Tennessee', '0500000US47043')] - data[('Dickson Incorporated', 'None')]
HumphInc = [data[('McEwen city, Tennessee', '1600000US4744840')],data[('New Johnsonville city, Tennessee', '1600000US4752820')],
            data[('Waverly city, Tennessee', '1600000US4778560')]]
data[('Humphreys Incorporated', 'None')] = sum(HumphInc)
data[('Humphreys Unincorporated', 'None')] = data[('Humphreys County, Tennessee', '0500000US47085')] - data[('Humphreys Incorporated', 'None')]
data[('Montgomery Incorporated', 'None')] = data[('Clarksville city, Tennessee', '1600000US4715160')]
data[('Montgomery Unincorporated', 'None')] = data[('Montgomery County, Tennessee', '0500000US47125')] - data[('Montgomery Incorporated', 'None')]

In [45]:
ok = data.transpose().reset_index()

In [46]:
ok.head()

,NAME,GEO_ID,agebysex_total_series,hhsize_avg,poverty_total_bysexbyage_series,poverty_belowlevel,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,tenure_total_series,tenure_owneroccunits,tenure_renteroccunits,units_total_series,units_one_detached,units_one_attached,units_two,units_threetofour,units_fivetonine,units_tentonineteen,units_twentytofortynine,units_fiftyormore,units_mobilehome,units_boatrvvanetc,hhtype_total_series,hhtype_familyhh,hhtype_familyhh_marriedcouplefam,hhtype_familyhh_otherfam,hhtype_familyhh_malenospouse,hhtype_familyhh_femalenospouse,hhtype_nonfamhh,hhtype_nonfamhh_householderalone,hhtype_nonfamhh_householdernotalone,hhincome_median,hhincome_total_series,hhincome_lessthan10000,hhincome_10to14999,hhincome_15to19999,hhincome_20to24999,hhincome_25to29999,hhincome_30to34999,hhincome_35to39999,hhincome_40to44999,hhincome_45to49999,hhincome_50to59999,hhincome_60to74999,hhincome_75to99999,hhincome_100to124999,hhincome125to149999,hhincome150to199999,hhincome200ormore,housingcost_total_selectedownercosts%hhincome_series,housingcost_total%ownercostwmortgage_series,housingcost_%ownercost30to34.9_wmortgage,housingcost_%ownercost35to39.9_wmortgage,housingcost_%ownercost40to49.9_wmortgage,housingcost_%ownercost50+_wmortgage,housingcost_total%ownercostwomortgage_series,housingcost_%ownercost30to34.9_womortgage,housingcost_%ownercost35to39.9_womortgage,housingcost_%ownercost40to49.9_womortgage,housingcost_%ownercost50+_womortgage,housingcost_total_rent%hhincome_series,housingcost_%rentercost30to34.9,housingcost_%rentercost35to39.9,housingcost_%rentercost40to49.9,housingcost_%rentercost50+,housingcost_medvalue_ownerocc_wmortgage,housingcost_medvalue_ownerocc_womortgage,housingcost_mediangrossrent_renteroccupied,commute_total_meansoftransportationtowork_series,commute_cartruckvan,commute_cartruckvan_drovealone,commute_cartruckvan_carpooled,commute_cartruckvan_carpooled_2ppl,commute_cartruckvan_carpooled_3ppl,commute_cartruckvan_carpooled_4ormoreppl,commute_publictransportation,commute_publictransportation_bus,commute_publictransportation_subwayorelevatedrail,commute_publictransportation_longdistancetrainorcommuterrail,commute_publictransportation_lightrailstreetcarortrolley,commute_publictransportation_ferryboat,commute_bicycle,commute_walk,commute_taxicabmotorcycleother,commute_workedfromhome,foreignborn_total,fb_europe,fb_asia,fb_africa,fb_oceania,fb_americas,fb_am_latinamerica,fb_am_northern,classworker_total_series,classworker_privateforprofit,classworker_privateforprofit_employee,classworker_privateforprofit_selfemployedincorporatedbusiness,classworker_privatenotforprofit,classworker_localgovt,classworker_stategovt,classworker_federalgovt,classworker_selfemployednonincorporatedbusiness,classworker_unpaidfamilyworker,lfstatus_total_sexbyagebyemploymentstatus16+_series,lfstatus_mciv_16to19,lfstatus_mciv_16to19_employed,lfstatus_mciv_16to19_unemployed,lfstatus_mciv_20to21,lfstatus_mciv_20to21_employed,lfstatus_mciv_20to21_unemployed,lfstatus_mciv_22to24,lfstatus_mciv_22to24_employed,lfstatus_mciv_22to24_unemployed,lfstatus_mciv_25to29,lfstatus_mciv_25to29_employed,lfstatus_mciv_25to29_unemployed,lfstatus_mciv_30to34,lfstatus_mciv_30to34_employed,lfstatus_mciv_30to34_unemployed,lfstatus_mciv_35to44,lfstatus_mciv_35to44_employed,lfstatus_mciv_35to44_unemployed,lfstatus_mciv_45to54,lfstatus_mciv_45to54_employed,lfstatus_mciv_45to54_unemployed,lfstatus_mciv_55to59,lfstatus_mciv_55to59_employed,lfstatus_mciv_55to59_unemployed,lfstatus_mciv_60to61,lfstatus_mciv_60to61_employed,lfstatus_mciv_60to61_unemployed,lfstatus_mciv_62to64,lfstatus_mciv_62to64_employed,lfstatus_mciv_62to64_unemployed,lfstatus_mciv_65to69,lfstatus_mciv_65to69_employed,lfstatus_mciv_65to69_unemployed,lfstatus_mciv_70to74,lfstatus_mciv_70to74_employed,lfstatus_mciv_70to74_unemployed,lfstatus_mciv_75+,lfstatus_mciv_75+_employed,lfstatus_mciv_75+_unemployed,lfstatus_fciv_16to19,lfstatus_fciv_16to19_employed,lfstatus_fciv_

In [47]:
ok.tail()

,NAME,GEO_ID,agebysex_total_series,hhsize_avg,poverty_total_bysexbyage_series,poverty_belowlevel,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,tenure_total_series,tenure_owneroccunits,tenure_renteroccunits,units_total_series,units_one_detached,units_one_attached,units_two,units_threetofour,units_fivetonine,units_tentonineteen,units_twentytofortynine,units_fiftyormore,units_mobilehome,units_boatrvvanetc,hhtype_total_series,hhtype_familyhh,hhtype_familyhh_marriedcouplefam,hhtype_familyhh_otherfam,hhtype_familyhh_malenospouse,hhtype_familyhh_femalenospouse,hhtype_nonfamhh,hhtype_nonfamhh_householderalone,hhtype_nonfamhh_householdernotalone,hhincome_median,hhincome_total_series,hhincome_lessthan10000,hhincome_10to14999,hhincome_15to19999,hhincome_20to24999,hhincome_25to29999,hhincome_30to34999,hhincome_35to39999,hhincome_40to44999,hhincome_45to49999,hhincome_50to59999,hhincome_60to74999,hhincome_75to99999,hhincome_100to124999,hhincome125to149999,hhincome150to199999,hhincome200ormore,housingcost_total_selectedownercosts%hhincome_series,housingcost_total%ownercostwmortgage_series,housingcost_%ownercost30to34.9_wmortgage,housingcost_%ownercost35to39.9_wmortgage,housingcost_%ownercost40to49.9_wmortgage,housingcost_%ownercost50+_wmortgage,housingcost_total%ownercostwomortgage_series,housingcost_%ownercost30to34.9_womortgage,housingcost_%ownercost35to39.9_womortgage,housingcost_%ownercost40to49.9_womortgage,housingcost_%ownercost50+_womortgage,housingcost_total_rent%hhincome_series,housingcost_%rentercost30to34.9,housingcost_%rentercost35to39.9,housingcost_%rentercost40to49.9,housingcost_%rentercost50+,housingcost_medvalue_ownerocc_wmortgage,housingcost_medvalue_ownerocc_womortgage,housingcost_mediangrossrent_renteroccupied,commute_total_meansoftransportationtowork_series,commute_cartruckvan,commute_cartruckvan_drovealone,commute_cartruckvan_carpooled,commute_cartruckvan_carpooled_2ppl,commute_cartruckvan_carpooled_3ppl,commute_cartruckvan_carpooled_4ormoreppl,commute_publictransportation,commute_publictransportation_bus,commute_publictransportation_subwayorelevatedrail,commute_publictransportation_longdistancetrainorcommuterrail,commute_publictransportation_lightrailstreetcarortrolley,commute_publictransportation_ferryboat,commute_bicycle,commute_walk,commute_taxicabmotorcycleother,commute_workedfromhome,foreignborn_total,fb_europe,fb_asia,fb_africa,fb_oceania,fb_americas,fb_am_latinamerica,fb_am_northern,classworker_total_series,classworker_privateforprofit,classworker_privateforprofit_employee,classworker_privateforprofit_selfemployedincorporatedbusiness,classworker_privatenotforprofit,classworker_localgovt,classworker_stategovt,classworker_federalgovt,classworker_selfemployednonincorporatedbusiness,classworker_unpaidfamilyworker,lfstatus_total_sexbyagebyemploymentstatus16+_series,lfstatus_mciv_16to19,lfstatus_mciv_16to19_employed,lfstatus_mciv_16to19_unemployed,lfstatus_mciv_20to21,lfstatus_mciv_20to21_employed,lfstatus_mciv_20to21_unemployed,lfstatus_mciv_22to24,lfstatus_mciv_22to24_employed,lfstatus_mciv_22to24_unemployed,lfstatus_mciv_25to29,lfstatus_mciv_25to29_employed,lfstatus_mciv_25to29_unemployed,lfstatus_mciv_30to34,lfstatus_mciv_30to34_employed,lfstatus_mciv_30to34_unemployed,lfstatus_mciv_35to44,lfstatus_mciv_35to44_employed,lfstatus_mciv_35to44_unemployed,lfstatus_mciv_45to54,lfstatus_mciv_45to54_employed,lfstatus_mciv_45to54_unemployed,lfstatus_mciv_55to59,lfstatus_mciv_55to59_employed,lfstatus_mciv_55to59_unemployed,lfstatus_mciv_60to61,lfstatus_mciv_60to61_employed,lfstatus_mciv_60to61_unemployed,lfstatus_mciv_62to64,lfstatus_mciv_62to64_employed,lfstatus_mciv_62to64_unemployed,lfstatus_mciv_65to69,lfstatus_mciv_65to69_employed,lfstatus_mciv_65to69_unemployed,lfstatus_mciv_70to74,lfstatus_mciv_70to74_employed,lfstatus_mciv_70to74_unemployed,lfstatus_mciv_75+,lfstatus_mciv_75+_employed,lfstatus_mciv_75+_unemployed,lfstatus_fciv_16to19,lfstatus_fciv_16to19_employed,lfstatus_fciv_

In [48]:
ok.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Columns: 548 entries, NAME to GeoFIPS
dtypes: float64(546), object(2)
memory usage: 385.4+ KB


In [49]:
ok.to_csv('../../data/ACS20105YR.csv', index = False)